In [25]:
import pandas as pd
import datetime as dt
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


In [3]:
df = pd.read_excel('Online_Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
#check the data
df.describe()

In [ ]:
df.info()

In [4]:
# Check for missing values in each column
missing_values = df.isnull().sum()
print(missing_values)

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [5]:
# Drop rows with missing CustomerID
df.dropna(subset=['CustomerID'], inplace=True)
# Remove rows with negative Quantity and Price
df= df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
#customerId change to integer
df['CustomerID'] = df['CustomerID'].astype(int)

In [6]:
#Compute Recency, Frequency,  Monetary value
snapshot_date = max(df['InvoiceDate']) + pd.DateOffset(days=1)

df['Total'] = df['Quantity'] * df['UnitPrice']


rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'Total': 'sum'
})
rfm.rename(columns={'InvoiceDate': 'Recency', 'InvoiceNo': 'Frequency', 'Total': 'MonetaryValue'}, inplace=True)

In [7]:
rfm.head(15)

,Recency,Frequency,MonetaryValue
CustomerID,,,
12346,326,1,77183.60
12347,2,7,4310.00
12348,75,4,1797.24
12349,19,1,1757.55
12350,310,1,334.40
12352,36,8,2506.04
12353,204,1,89.00
12354,232,1,1079.40
12355,214,1,459.40


In [8]:
#summary stats or describe the data
rfm.describe()

#Find the customer with the highest Monetaryvalue
dis=rfm.sort_values(by='MonetaryValue',ascending=False)
dis.head()
#its CustomerID 14646

,Recency,Frequency,MonetaryValue
CustomerID,,,
14646,2,73,280206.02
18102,1,60,259657.30
17450,8,46,194550.79
16446,1,2,168472.50
14911,1,201,143825.06


In [27]:
#what did Customer ID 14646 buy, lets take a look
Customer_stats= df.loc[df['CustomerID'] == 14646]
Customer_stats


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total
37952,539491,21981,PACK OF 12 WOODLAND TISSUES,12,2010-12-20 10:09:00,0.29,14646,Netherlands,3.48
37953,539491,21986,PACK OF 12 PINK POLKADOT TISSUES,12,2010-12-20 10:09:00,0.29,14646,Netherlands,3.48
37954,539491,22720,SET OF 3 CAKE TINS PANTRY DESIGN,2,2010-12-20 10:09:00,4.95,14646,Netherlands,9.90
37955,539491,21931,JUMBO STORAGE BAG SUKI,1,2010-12-20 10:09:00,1.95,14646,Netherlands,1.95
37956,539491,22613,PACK OF 20 SPACEBOY NAPKINS,2,2010-12-20 10:09:00,0.85,14646,Netherlands,1.70
...,...,...,...,...,...,...,...,...,...
534958,581176,22908,PACK OF 20 NAPKINS RED APPLES,96,2011-12-07 15:19:00,0.72,14646,Netherlands,69.12
534959,581176,22907,PACK OF 20 NAPKINS PANTRY DESIGN,96,2011-12-07 15:19:00,0.72,14646,Netherlands,69.12
534960,581176,22029,SPACEBOY BIRTHDAY CARD,72,2011-12-07 15:19:00,0.36,14646,Netherlands,25.92
534961,581176,22712,CARD DOLLY GIRL,72,2011-12-07 15:19:00,0.36,14646,Netherlands,25.92
